## Imports

In [125]:
from numpy.random import seed
seed(888)
import tensorflow as tf
tf.random.set_seed(404)
import os
import numpy as np

from time import strftime
from PIL import Image

## Constants 

In [126]:
X_TRAIN_PATH = 'MNIST/digit_xtrain.csv'
X_TEST_PATH = 'MNIST/digit_xtest.csv'
Y_TRAIN_PATH = 'MNIST/digit_ytrain.csv'
Y_TEST_PATH = 'MNIST/digit_ytest.csv'

LOGGING_PATH = 'tensorboard_mnist_digit_logs/'

NR_CLASSES = 10
VALIDATION_SIZE = 10000
IMAGE_WIDTH = 28
IMAGE_HEIGHT = 28
CHANNELS = 1
TOTAL_INPUTS = IMAGE_WIDTH*IMAGE_HEIGHT*CHANNELS

## Get the Data


In [127]:
%%time 
y_train_all = np.loadtxt(Y_TRAIN_PATH, delimiter=',', dtype=int)

CPU times: total: 15.6 ms
Wall time: 8.99 ms


In [128]:
y_test = np.loadtxt(Y_TEST_PATH, delimiter=',', dtype=int)

In [129]:
x_train_all = np.loadtxt(X_TRAIN_PATH, delimiter=',', dtype=int)

In [130]:
x_test = np.loadtxt(X_TEST_PATH, delimiter=',', dtype=int)

## Explore

In [131]:
# Output: 60,000 examples with 784 values per example
x_train_all.shape


(60000, 784)

In [132]:
x_train_all[0]

array([  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,   0,   0,   3,  18,  18,  18,
       126, 136, 175,  26, 166, 255, 247, 127,   0,   0,   0,   0,   0,
         0,   0,   0,   0,   0,   0,   0,  30,  36,  94, 154, 17

In [133]:
y_train_all.shape

(60000,)

In [134]:
x_test.shape

(10000, 784)

In [135]:
# Categories or classes of images
y_train_all[:5]

array([5, 0, 4, 1, 9])

## Data Preprocessing

In [136]:
# Re-scale by the largest value in he data
x_train_all, x_test = x_train_all / 255.0 , x_test / 255.0


In [137]:
values = y_train_all[:5]

# Return a 2-D array with ones on the diagonal and zeros elsewhere.
# Each number in the index array indicates which value in the preceding 
# array to use in the place of the index.  

# Using an array as an index to pull out several rows from the identity matrix
np.eye(10)[values] 

array([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 0., 1.]])

In [138]:
np.eye(10)[2]

array([0., 0., 1., 0., 0., 0., 0., 0., 0., 0.])

### Convert target values to one-hot encoding

In [139]:
y_train_all = np.eye(NR_CLASSES)[y_train_all]

In [140]:
y_train_all.shape

(60000, 10)

In [141]:
y_test = np.eye(NR_CLASSES)[y_test]
y_test.shape

(10000, 10)

### Create validation dataset from training data

**Challenge:** Split the training dataset into a smaller training dataset and a validation dataset for the features and the labels. 
Create four arrays" ```x_val, y_val, x_train, and y_train``` from ```x_train_all``` and ```y_train_all```. Use the validation size of 10,000. 

In [142]:
x_val = x_train_all[:VALIDATION_SIZE]
y_val = y_train_all[:VALIDATION_SIZE]


In [143]:
x_train = x_train_all[VALIDATION_SIZE:]
y_train = y_train_all[VALIDATION_SIZE:]

In [144]:
x_train.shape

(50000, 784)

In [145]:
x_val.shape

(10000, 784)

# Setup TensorFlow Graph

In [146]:
tf.compat.v1.disable_eager_execution()

X = tf.compat.v1.placeholder(tf.float32, shape=[None, TOTAL_INPUTS], name='X') # Data has 784 features
Y = tf.compat.v1.placeholder(tf.float32, shape=[None, NR_CLASSES], name='labels') # 10 = Number of categories. 

### Neural Network Architecture
#### Hyperparameters 

In [147]:
nr_epochs = 50
learning_rate = 1e-3 #(0.001)

# How many neurons per layer
n_hidden1= 512
n_hidden2 = 64


## Code for first part of module. 
## Created a function for efficiency

In [148]:
# with tf.name_scope('hidden_1'):
#     # Initial weights in the first hidden layer
#     # Input shape = [inputs, neurons]
#     initial_w1 = tf.random.truncated_normal(shape=[TOTAL_INPUTS, n_hidden1], stddev=0.1, seed=42) 

#     # This holds the weights
#     w1 = tf.Variable(initial_value=initial_w1, name='w1')

#     # Biases will start with the same value 
#     initial_b1 = tf.constant(value=0.0, shape=[n_hidden1])
#     b1 = tf.Variable(initial_value=initial_b1, name='b1')

#     # Matrix Multiplication = matmul
#     layer1_in = tf.matmul(X, w1) + b1

#     layer1_out = tf.nn.relu(layer1_in)

**Challenge:** Set up the second hidden layer. This layer has 64 neurons and needs to work off the output of the first hidden layer (see above). Then setup the output layer. Remember, the output layer will use the softmax activation function. **EACH LAYER SEEMS TO HAVE TO BE INITIALIZED**

In [149]:
# with tf.name_scope('hidden_2'):

#     # Initial weights in the second hidden layer
#     # Input shape = [inputs, neurons]
#     initial_w2 = tf.random.truncated_normal(shape=[n_hidden1, n_hidden2], stddev=0.1, seed=42) 

#     # This holds the weights
#     w2 = tf.Variable(initial_value=initial_w2, name='w2')

#     # Biases will start with the same value 
#     initial_b2 = tf.constant(value=0.0, shape=[n_hidden2])
#     b2 = tf.Variable(initial_value=initial_b2, name='b2')

#     layer2_in = tf.matmul(layer1_out, w2) + b2
#     layer2_out = tf.nn.relu(layer2_in)  

In [150]:
# with tf.name_scope('output_layer'):

#     # Initial weights in the third hidden layer
#     # Input shape = [inputs, neurons]
#     initial_w3 = tf.random.truncated_normal(shape=[n_hidden2, NR_CLASSES], stddev=0.1, seed=42) 

#     # This holds the weights
#     w3 = tf.Variable(initial_value=initial_w3, name='w3')

#     # Biases will start with the same value 
#     initial_b3 = tf.constant(value=0.0, shape=[NR_CLASSES])
#     b3 = tf.Variable(initial_value=initial_b3, name='b3')

#     layer3_in = tf.matmul(layer2_out, w3) + b3
#     output = tf.nn.softmax(layer3_in)

In [151]:
def setup_layer(input, weight_dim, bias_dim, name):
    with tf.name_scope(name): 
        # Initial weights in the third hidden layer
        # Input shape = [inputs, neurons]
        initial_w = tf.random.truncated_normal(shape=weight_dim, stddev=0.1, seed=42) 
        # This holds the weights
        w = tf.Variable(initial_value=initial_w, name='W')

        # Biases will start with the same value 
        initial_b = tf.constant(value=0.0, shape=bias_dim)
        b = tf.Variable(initial_value=initial_b, name='B')

        layer_in = tf.matmul(input, w) + b

        if name=='out': 
            layer_out = tf.nn.softmax(layer_in)
        else:
            layer_out = tf.nn.relu(layer_in)

        tf.compat.v1.summary.histogram('weights', w)
        tf.compat.v1.summary.histogram('biases', b)  

        return layer_out


In [152]:
# layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
#                       bias_dim=[n_hidden1], name='layer_1')

# layer_2 = setup_layer(layer_1, weight_dim=[n_hidden1, n_hidden2],
#                       bias_dim=[n_hidden2], name='layer_2')

# output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
#                       bias_dim=[NR_CLASSES], name='out')

# model_name = f'{n_hidden1}-{n_hidden2} LR{learning_rate} E{nr_epochs}'

In [153]:
layer_1 = setup_layer(X, weight_dim=[TOTAL_INPUTS, n_hidden1],
                      bias_dim=[n_hidden1], name='layer_1')

layer_drop = tf.nn.dropout(layer_1, rate=0.2, name='dropout_layer')

layer_2 = setup_layer(layer_drop, weight_dim=[n_hidden1, n_hidden2],
                      bias_dim=[n_hidden2], name='layer_2')

output = setup_layer(layer_2, weight_dim=[n_hidden2, NR_CLASSES],
                      bias_dim=[NR_CLASSES], name='out')

model_name = f'{n_hidden1}-DO-{n_hidden2} LR{learning_rate} E{nr_epochs}'

## Tensorboard Setup

In [154]:
# Folder for Tensorboard

folder_name = f'{model_name} at {strftime("%H%M")}'
directory = os.path.join(LOGGING_PATH, folder_name)

try:
    os.makedirs(directory)
except OSError as exception:
    print(exception.strerror)
else: 
    print('Succesfully created directories.')

Succesfully created directories.


# Loss, Optimization, & Metrics

##### Defining Loss Function

In [155]:
with tf.name_scope('loss_calc'):
 
    # Gets average because we are training in batches. 
    loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output))

##### Defining Optimizer

In [156]:
with tf.name_scope('optimizer'):
 
    optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    train_step = optimizer.minimize(loss) 

##### Accuracy Metric

In [157]:
with tf.name_scope('accuracy_calc'):

    correct_prediction = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    # correct_pred = tf.equal(tf.argmax(output, axis=1), tf.argmax(Y, axis=1))
    # accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [158]:
with tf.name_scope('performance'):

    tf.compat.v1.summary.scalar('accuracy', accuracy)
    tf.compat.v1.summary.scalar('loss', loss)

## Check Input Images in Tensorboard

In [159]:
with tf.name_scope('show_image'):
    x_image = tf.reshape(X, [-1,28,28,1])
    tf.compat.v1.summary.image('image_input', x_image, max_outputs=4)

## Run Session

In [160]:
sess = tf.compat.v1.Session()

#### Setup Filewriter and Merge Summaries

In [161]:
merged_summary = tf.compat.v1.summary.merge_all()


In [162]:
train_writer = tf.compat.v1.summary.FileWriter(directory + '/train')
train_writer.add_graph(sess.graph) 

validation_writer = tf.compat.v1.summary.FileWriter(directory + '/validation')

# Initialize all the variables

In [163]:
init = tf.compat.v1.global_variables_initializer()
sess.run(init)

In [164]:
# b3.eval(sess)

##### Batching the Data
###### Next step is to split training data to batches with 1000 samples each

In [165]:
size_of_batch = 1000

In [166]:
num_examples = y_train.shape[0]
nr_iterations = int(num_examples/size_of_batch)

index_in_epoch = 0

In [167]:
def next_batch(batch_size, data, labels):
    
    # we can only access the global variable but cannot modify it 
    # from inside the function.
    global num_examples
    global index_in_epoch

    start = index_in_epoch
    index_in_epoch += batch_size

    if index_in_epoch > num_examples:
        start = 0 
        index_in_epoch = batch_size

    end = index_in_epoch

    return data[start:end], labels[start:end]



In [168]:
print(type(merged_summary))

<class 'tensorflow.python.framework.ops.SymbolicTensor'>


##### Training Loop

In [169]:
# Ensure that all necessary variables and operations are defined and initialized before this code block

for epoch in range(nr_epochs):

    # ============= Training Dataset =========
    for i in range(nr_iterations):
        
        # Assuming next_batch is a function that fetches the next batch of data
        batch_x, batch_y = next_batch(batch_size=size_of_batch, data=x_train, labels=y_train)

        feed_dictionary = {X:batch_x, Y:batch_y}

        sess.run(train_step, feed_dict=feed_dictionary)

    # Fetching summary and accuracy after each epoch
    s, batch_accuracy = sess.run(fetches=[merged_summary, accuracy], feed_dict=feed_dictionary)

    # Adding summary to the writer
    train_writer.add_summary(s, epoch)

    # Printing the training accuracy for the current epoch
    print(f'Epoch {epoch} \t| Training Accuracy = {batch_accuracy}')

    # ================= Validation ================
    summary = sess.run(fetches=merged_summary, feed_dict={X:x_val, Y:y_val})
    validation_writer.add_summary(summary, epoch)

print('Done Training.')


Epoch 0 	| Training Accuracy = 0.8619999885559082
Epoch 1 	| Training Accuracy = 0.8809999823570251
Epoch 2 	| Training Accuracy = 0.9200000166893005
Epoch 3 	| Training Accuracy = 0.9629999995231628
Epoch 4 	| Training Accuracy = 0.9739999771118164
Epoch 5 	| Training Accuracy = 0.9750000238418579
Epoch 6 	| Training Accuracy = 0.9779999852180481
Epoch 7 	| Training Accuracy = 0.9789999723434448
Epoch 8 	| Training Accuracy = 0.9800000190734863
Epoch 9 	| Training Accuracy = 0.9789999723434448
Epoch 10 	| Training Accuracy = 0.9810000061988831
Epoch 11 	| Training Accuracy = 0.9850000143051147
Epoch 12 	| Training Accuracy = 0.9860000014305115
Epoch 13 	| Training Accuracy = 0.9860000014305115
Epoch 14 	| Training Accuracy = 0.9869999885559082
Epoch 15 	| Training Accuracy = 0.9890000224113464
Epoch 16 	| Training Accuracy = 0.9900000095367432
Epoch 17 	| Training Accuracy = 0.9860000014305115
Epoch 18 	| Training Accuracy = 0.9900000095367432
Epoch 19 	| Training Accuracy = 0.9900000

# Make a Prediction

In [170]:
# from PIL import Image
img = Image.open('MNIST/test_img.png')
img

In [171]:
# convert to grayscale
bw = img.convert(mode='L')

In [172]:
img_array = np.invert(bw)

In [173]:
img_array.shape

(28, 28)

In [174]:
test_img = img_array.ravel()

In [175]:
test_img.shape

(784,)

In [176]:
# Output = fetches
# Axis = 1 = row
prediction = sess.run(fetches=tf.argmax(output, axis=1), feed_dict={X: [test_img]})

In [177]:
print(f'Prediction for test image is {prediction}')

Prediction for test image is [2]


# Testing and Evaluation

**Challenge:** Calculate the accuracy over the test dataset (```x_test``` and ```y_test```). Use your knowledge of running a session to get the accuracy. Display the accuracy as a percentage rounded to two decimal numbers. 

In [178]:
test_accuracy = sess.run(fetches=accuracy, feed_dict={X:x_test, Y:y_test})
print(f'Accuracy on test set is {test_accuracy:0.2%}')

Accuracy on test set is 97.75%


## Reset for the Next Run

In [179]:
train_writer.close()
validation_writer.close()
sess.close()
tf.compat.v1.reset_default_graph()

### EXPERIMENTAL TENSORBOARD ON NOTEBOOK

In [180]:
# %load_ext tensorboard


In [181]:
# %tensorboard --logdir logs